### Building Chatbot with multiple tool using langraph

### Aim
Create a chatbot with tool capabilities from arxiv, wikipedia search and some functions

In [ ]:
from langchain_community.tools import WikipediaQueryRun,ArxivQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

### Arxiv Wrapper

In [ ]:
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=2,doc_content_chars_max=500)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)

In [ ]:
arxiv.invoke("Attention is all you need")

### Wekipedia Wrapper

In [ ]:
api_wrapper_wikipedia = WikipediaAPIWrapper(top_k_results=3,doc_content_chars_max=1000)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wikipedia)

In [ ]:
wiki.invoke("What is Generative AI")

In [ ]:
from dotenv import load_dotenv
load_dotenv()
import os

os.environ['TAVILY_API_KEY'] = os.getenv("TAVILY_API_KEY")
os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")

### Tavily Search Tool

In [ ]:
from langchain_tavily import TavilySearch
tavily_tool = TavilySearch(max_result=5,topic="general")

In [ ]:
tavily_tool.invoke("Provide me the latest news about Indian Politics")

In [ ]:
## Combine all the tools in the list
tools = [arxiv,wiki,tavily_tool]


### Initialize my LLm model and binding it with tools

In [ ]:
from langchain_groq import ChatGroq
llm=ChatGroq(model="llama-3.1-8b-instant")
llm_with_tools=llm.bind_tools(tools)

In [ ]:
from pprint import pprint
from langchain_core.messages import HumanMessage,AIMessage

llm_with_tools.invoke([HumanMessage(content="What is the recent AI news")])

In [ ]:
llm_with_tools.invoke([HumanMessage(content="What is the recent AI news")]).tool_calls

### Entire CHatbot with langgraph

In [ ]:
# State Schema
from typing_extensions import TypedDict
from langchain_core.messages import AnyMessage
from typing import Annotated
from langgraph.graph import add_messages

class State(TypedDict):
    messages:Annotated[list[AnyMessage],add_messages]

In [ ]:
from IPython.display import Image,display
from langgraph.graph import StateGraph,START,END
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition

## Node definition
def tool_calling_llm(state:State):
    return {"messages":[llm_with_tools.invoke(state['messages'])]}

# Build graph 
builder = StateGraph(State)

# add node
builder.add_node("tool_calling_llm",tool_calling_llm)
builder.add_node('tools',ToolNode(tools))

#add edges
builder.add_edge(START,'tool_calling_llm')
builder.add_conditional_edges("tool_calling_llm",tools_condition)
builder.add_edge("tools",END)

#compile 
graph = builder.compile()

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
from pprint import pprint
messages = graph.invoke({"messages":HumanMessage(content="What is the attention is all you need")})
for m in messages['messages']:
    m.pretty_print()

In [ ]:
from pprint import pprint
messages = graph.invoke({"messages":HumanMessage(content="what is the current condition of Nepal")})
for m in messages['messages']:
    m.pretty_print()

In [ ]:
from pprint import pprint
messages = graph.invoke({"messages":HumanMessage(content="What is Machine Learning")})
for m in messages['messages']:
    m.pretty_print()

In [ ]:
from pprint import pprint
messages = graph.invoke({"messages":HumanMessage(content="What is MAchine Learning.Give answer from wikipedia")})
for m in messages['messages']:
    m.pretty_print()

- First we make the tools for wikipedia and arxiv using wikipedia/arxiv wrapper and query_run
- Then for internet search we use tavily_tool from langchain_tavily
- Combine all the tools in list form
- Initialize the Groq model and bind it with tool
- Creating the State schema and node definition
- create the node (calling_with_llm, tool with the help to ToolNode)
- add the edge (for adding tools we use tools_condition)
- display the graph
- query with the model